In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle


from sklearn.preprocessing import LabelEncoder
from itertools import product

# Описание

## Описание датасетов

*   sales_train.csv - тренировочная база данных. Ежедневный отчет по продажам с Января 2013 по Октябрь 2015
*   test.csv - тестовая база данных. Необходимо предсказать продажи для магазинов и товаров на Ноябрь 2015
*   sample_submission.csv - образец файла решения
*   items.csv - информация про товары
*   item_categories.csv  - информация про категории товаров
*   shops.csv- информация о магазинах

## Описание полей базы данных

*   ID - код
*   shop_id - код магазина
*   item_id - код товара
*   item_category_id - код категории товара
*   item_cnt_day - количество проданного товара за день. Мы предсказываем данный показатель за месяц
*   item_price - текущая цена на товар
*   date - дата в формате день/месяц/год (дд/мм/гггг)
*   date_block_num - счетчик месяцев, Январь 2013 - 0, Октябрь 2015 - 33
*   item_name - наименование товара
*   shop_name - название магазина
*   item_category_name - название категории 

# Загрузка данных

In [1]:
df_test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
df_item_categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
df_items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
df_shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
df_sales_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')

Из датасета с тренировачными данными, убираем магазины и предметы которых нет в датасете для предсказанияДанным действием мы ощутимо облегачаем работу алгоритмам. Данным действием мы ощутимо облегачаем работу алгоритмам.

In [1]:
df_sales_train = df_sales_train.loc[df_sales_train["shop_id"].isin(df_test["shop_id"].unique()), :]
df_sales_train = df_sales_train.loc[df_sales_train["item_id"].isin(df_test["item_id"].unique()), :]

Информация о датасетах

In [1]:
# Товары
df_items.info()

In [1]:
# Магазины
df_shops.info()

In [1]:
# Категории товаров
df_item_categories.info()

In [1]:
# Тренировачный датасет с продажами
df_sales_train.info()

In [1]:
# Тестовый датасет к которому предсказываем продажи
df_test.info()

# Выбросы

In [1]:
plt.figure(figsize=(10,4))
plt.xlim(-100, 3000)
sns.boxplot(x = df_sales_train.item_cnt_day)

plt.figure(figsize=(10,4))
sns.boxplot(x =df_sales_train.item_price)

Замечаем выбросы по дням продажи и по цене товара и убираем их

In [1]:
df_sales_train = df_sales_train[df_sales_train.item_price<50000]
df_sales_train = df_sales_train[df_sales_train.item_cnt_day<2000]

Просматриваем график после обработки, чтобы убедиться что убрали все выбросы

In [1]:
plt.figure(figsize=(10,4))
plt.xlim(-100, 3000)
sns.boxplot(x = df_sales_train.item_cnt_day)

plt.figure(figsize=(10,4))
sns.boxplot(x =df_sales_train.item_price)

# Предварительный анализ данных

In [1]:
matrix = []
group = df_sales_train .groupby(['date_block_num','shop_id','item_id']).agg({'item_price':['last'],'item_cnt_day': ['sum']})
group.reset_index(inplace=True)
group.columns = ['date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_month']

matrix = pd.merge(group, df_items, on='item_id', how='left')
matrix = pd.merge(matrix, df_shops, on='shop_id', how='left')
matrix = pd.merge(matrix, df_item_categories, on='item_category_id', how='left')

matrix.loc[matrix.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
matrix['city'] = matrix['shop_name'].str.split(' ').map(lambda x: x[0])
matrix.loc[matrix.city == '!Якутск', 'city'] = 'Якутск'
matrix['city_code'] = LabelEncoder().fit_transform(matrix['city'])
matrix.head()
matrix.head()

Рассмотрим продажи в зависимости от категории и имени товара.

In [1]:
plt.figure(figsize=(30,3))
sns.barplot(data=matrix.groupby(by='item_category_name').sum().reset_index(), x="item_category_name", y='item_cnt_month')
plt.xticks(rotation=90)
plt.show()

Можно заметить что продажи имеют тесную связь с категорией товара, но мы не можем посмотреть на это отдельно, поэтому нам стоит разделить категорию и наименовании товара на отдельные признки. Чем мы займем в разделе Feature Engineering.

Логически можно предположить, что продажи могут иметь связь с месяцем продажи, так как, например к новому году люди начинают активно закупать подарки, за счет чего продажи растут. Посмотрим на графике

In [1]:
plt.figure(figsize=(30,3))
sns.barplot(data=matrix.groupby(by='date_block_num').sum().reset_index(), x="date_block_num", y='item_cnt_month')
plt.xticks(rotation=0)
plt.show()

Нумерация месяцев начинается с 0, следовательно 11 и 23 месяц - Декабрь. Действительно есть связь между продажами и месяцами.

Одна из самых очевидных зависмостей по нашему мнению - зависимость продаж от цены. Посмотрим на графике.

In [1]:
fig, axes = plt.subplots(figsize=(25,8))
sns.lineplot(data=matrix.groupby(by='item_price').sum().reset_index(), x="item_price", y='item_cnt_month', ax=axes)

Что и требовалось доказать. Товары до 8000 самые привлекательные для покупки потребителей. Можно сразу сказать что будет сильная зависимость данных признаков

Можем предположить зависимость от географических покзаателей, от города. Давайте посмотрим детальнее.

In [1]:
plt.figure(figsize=(30,3))
sns.barplot(data=matrix.groupby(by='city').sum().reset_index(), x="city", y='item_cnt_month')
plt.xticks(rotation=30)
plt.show()

Москва занимает лидирующую позицию, она является столицой и самым населенным городом, но как нам кажется данный признак имеет не самое большое влияние на продажи

# Feature Engineering

## Предварительная обработка данных по магазинам, категориям товаров и товарам, тестовый датасет

### Обработка магазинов 

Добавляем в датафрейм магазинов столбец "city" (Город). Для полноценного считывания города Сергиев Посад, убираем пробел, так как маппинг идет по пробелам

In [1]:
df_shops.loc[df_shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
df_shops['city'] = df_shops['shop_name'].str.split(' ').map(lambda x: x[0])
df_shops.loc[df_shops.city == '!Якутск', 'city'] = 'Якутск'
df_shops['city_code'] = LabelEncoder().fit_transform(df_shops['city'])
df_shops.head()

Сразу убираем текстовые названия, в процессе обучения они будут нам не нужны

In [1]:
df_shops.drop(['shop_name','city'], axis=1, inplace=True)
df_shops.head()

### Обработка категорий

Добавляем к датафрейму категорий столбцы split (разделение), type (тип категории), type_code (код типа категории), suptype (тип подкатегории), suptype_code (код типа подкатегории)

In [1]:
df_item_categories['split'] = df_item_categories['item_category_name'].str.split('-')
df_item_categories['type'] = df_item_categories['split'].map(lambda x: x[0].strip())
df_item_categories['type_code'] = LabelEncoder().fit_transform(df_item_categories['type'])
# Если подкатегория равна пустому значению 
df_item_categories['subtype'] = df_item_categories['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
df_item_categories['subtype_code'] = LabelEncoder().fit_transform(df_item_categories['subtype'])
cats = df_item_categories[['item_category_id','type_code', 'subtype_code']]
df_item_categories.head()

Сразу убираем текстовые названия, в процессе обучения они будут нам не нужны

In [1]:
df_item_categories.drop(['item_category_name', 'split', 'type', 'subtype'],axis=1, inplace=True)

In [1]:
df_item_categories.head()

### Обработка товаров

In [1]:
df_items.head()

Сразу убираем текстовые названия, в процессе обучения они будут нам не нужны

In [1]:
df_items.drop(['item_name'], axis=1, inplace=True)
df_items.head()

### Обработка тестовых данных

Добавим в тестовый датасет для Нобря 2015 колонку date_block_num c индексом месяцы (34)

In [1]:
df_test['date_block_num'] = 34
df_test['date_block_num'] = df_test['date_block_num'].astype(np.int8)
df_test['shop_id'] = df_test['shop_id'].astype(np.int8)
df_test['item_id'] = df_test['item_id'].astype(np.int16)

In [1]:
df_test.head()

## Подготовка данных: магазины, товары и категории в тренировочный датасет и тестовый датасет

matrix - матрица которую мы подготавливаем для обучения модели, за счет присоединения признаков и разных датасетов загруженных вначале

In [1]:
matrix = []

for month in df_sales_train['date_block_num'].unique():
    shops_in_month = df_sales_train.loc[df_sales_train['date_block_num']==month, 'shop_id'].unique()
    items_in_month = df_sales_train.loc[df_sales_train['date_block_num']==month, 'item_id'].unique()
    matrix.append(np.array(list(product(*[[month], shops_in_month, items_in_month])), dtype='int32'))

# Благодаря функции product перебираем возможные комбинации

matrix = np.vstack(matrix)
matrix = pd.DataFrame(matrix, columns = ['date_block_num','shop_id', 'item_id', ])

In [1]:
matrix = pd.concat(
    [matrix, df_test.drop(['ID'], axis=1)],
    ignore_index=True, sort=False, keys= ['date_block_num','shop_id', 'item_id' ]
)
matrix

In [1]:
group = df_sales_train.groupby(['date_block_num','shop_id','item_id']).agg({'item_price':['last'],'item_cnt_day': ['sum']})
group.reset_index(inplace=True)
group.columns = ['date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_month']

Заполняем колонку проданного товара за месяц - item_cnt_month, если получаем пустое значение то ставим 0

In [1]:
matrix = pd.merge(matrix, group, on= ['date_block_num','shop_id', 'item_id'], how='left')
matrix['item_cnt_month'] = (matrix['item_cnt_month']
                                .fillna(0)
                                .clip(0,20) 
                                .astype(np.float16))

In [1]:
matrix = pd.merge(matrix, df_items, on='item_id', how='left')
matrix = pd.merge(matrix, df_shops, on='shop_id', how='left')
matrix = pd.merge(matrix, df_item_categories, on='item_category_id', how='left')

matrix

## Целевые задержки

Сформируем целевые задержки по последним трём месяцам, за полгода и за год

In [1]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [1]:
matrix = lag_feature(matrix, [1,2,3,6,12], 'item_cnt_month')

In [1]:
matrix.columns

In [1]:
matrix.info()

## Сдвиги значений по одному и нескольким признакам 

Сдвиг среднемесячных продаж

In [1]:
group = matrix.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num'], how='left')
matrix['date_avg_item_cnt'] = matrix['date_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_avg_item_cnt')
matrix.drop(['date_avg_item_cnt'], axis=1, inplace=True)

Сдвиг среднемесячных продаж по товарам

In [1]:
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_cnt'] = matrix['date_item_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,6,12], 'date_item_avg_item_cnt')
matrix.drop(['date_item_avg_item_cnt'], axis=1, inplace=True)

Сдвиг среднемесячных продаж по магазинам

In [1]:
group = matrix.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shop_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_avg_item_cnt'] = matrix['date_shop_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,6,12], 'date_shop_avg_item_cnt')
matrix.drop(['date_shop_avg_item_cnt'], axis=1, inplace=True)

Сдвиг среднемесячных продаж по категориям товаров

In [1]:
group = matrix.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_category_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_category_id'], how='left')
matrix['date_category_avg_item_cnt'] = matrix['date_category_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_category_avg_item_cnt')
matrix.drop(['date_category_avg_item_cnt'], axis=1, inplace=True)

Сдвиг среднемесячных продаж по магазинам и категориям товаров

In [1]:
group = matrix.groupby(['date_block_num', 'shop_id', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_cat_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
matrix['date_shop_cat_avg_item_cnt'] = matrix['date_shop_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_cat_avg_item_cnt')
matrix.drop(['date_shop_cat_avg_item_cnt'], axis=1, inplace=True)

Сдвиг среднемесячных продаж по магазинам и типу товаров

In [1]:
group = matrix.groupby(['date_block_num', 'shop_id', 'type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_type_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'type_code'], how='left')
matrix['date_shop_type_avg_item_cnt'] = matrix['date_shop_type_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_type_avg_item_cnt')
matrix.drop(['date_shop_type_avg_item_cnt'], axis=1, inplace=True)

Сдвиг среднемесячных продаж по магазинам и подтипу товара

In [1]:
group = matrix.groupby(['date_block_num', 'shop_id', 'subtype_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_subtype_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'subtype_code'], how='left')
matrix['date_shop_subtype_avg_item_cnt'] = matrix['date_shop_subtype_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_subtype_avg_item_cnt')
matrix.drop(['date_shop_subtype_avg_item_cnt'], axis=1, inplace=True)

Сдвиг среднемесячных продаж по городам

In [1]:
group = matrix.groupby(['date_block_num', 'city_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'city_code'], how='left')
matrix['date_city_avg_item_cnt'] = matrix['date_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_city_avg_item_cnt')
matrix.drop(['date_city_avg_item_cnt'], axis=1, inplace=True)

Сдвиг среднемесячных продаж по городам и товарам

In [1]:
group = matrix.groupby(['date_block_num', 'item_id', 'city_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_id', 'city_code'], how='left')
matrix['date_item_city_avg_item_cnt'] = matrix['date_item_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_item_city_avg_item_cnt')
matrix.drop(['date_item_city_avg_item_cnt'], axis=1, inplace=True)

Сдвиг среднемесячных продаж по типу товара

In [1]:
group = matrix.groupby(['date_block_num', 'type_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_type_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'type_code'], how='left')
matrix['date_type_avg_item_cnt'] = matrix['date_type_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_type_avg_item_cnt')
matrix.drop(['date_type_avg_item_cnt'], axis=1, inplace=True)

Сдвиг среднемесячных продаж по подтипу товара

In [1]:
group = matrix.groupby(['date_block_num', 'subtype_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_subtype_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'subtype_code'], how='left')
matrix['date_subtype_avg_item_cnt'] = matrix['date_subtype_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_subtype_avg_item_cnt')
matrix.drop(['date_subtype_avg_item_cnt'], axis=1, inplace=True)

In [1]:
matrix.columns

In [1]:
matrix.info()

## Тренд цены

In [1]:
group = df_sales_train.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['item_id'], how='left')
matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)

group = df_sales_train.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace=True)

In [1]:
matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

lags = [1,2,3,4,5,6]
matrix = lag_feature(matrix, lags, 'date_item_avg_item_price')

Последнии продажи магазина и товара

In [1]:
cache = {}
matrix['item_shop_last_sale'] = -1
matrix['item_shop_last_sale'] = matrix['item_shop_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = str(row.item_id)+' '+str(row.shop_id)
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        matrix.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        cache[key] = row.date_block_num 

In [1]:
cache = {}
matrix['item_last_sale'] = -1
matrix['item_last_sale'] = matrix['item_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = row.item_id
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        if row.date_block_num>last_date_block_num:
            matrix.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
            cache[key] = row.date_block_num   

Первые продажи магазина и товара

In [1]:
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

Заполние пустых ячеек связнных с залдержками и проданнами товарами за месяц

In [1]:
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)         
    return df

matrix = fill_na(matrix)

In [1]:
matrix.info()

# CATBoost

Выбор CATBoost'а обусловлен его скоростью, проанализировав статьи, посмотерв лекции, принято решение попробовать его. Так же пробовали обучать модель на XGBoost, но выходило долго - 20+ минут, в то время как CATBoost справляется за 2 минуты, не теряя сильно в точности. Данные тесты мы убрали, так как они не являются главное целью нашего проекта. Так же CATBoost лояльно относиться к проускам, многие из них нам нет большой нужды заполнять.

## Немного подготовки

Сохраняем сделанную матрицу

In [1]:
matrix.to_pickle('data.pkl')

In [1]:
with open('test.pkl', 'wb') as f:
...     pickle.dump(df_test, f)

Освобождаем ресурсы проекта 

In [1]:
%reset -f

Заново импортируем нужные библиотеки

In [1]:
!pip install catboost

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
import catboost as ctb
import numpy as np

from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error as mse

Считываем нашу сохраненную матрицу

In [1]:
data = pd.read_pickle('data.pkl')
test = pd.read_pickle('test.pkl')

Разделяем данные

In [1]:
# Тренировочная выборка
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
# Валидационная выборка
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
# Тестовая выборка
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

## Учимся!

In [1]:
model = CatBoostRegressor(
    max_depth=10, 
    loss_function='RMSE',
    n_estimators = 1000,
    eta = 0.3,
    random_seed=123, 
    )
model.fit(
    X_train, 
    Y_train, 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)],  
    verbose=True, 
    #early_stopping_rounds = 20
    )

## Предсказываем валидационные значения

In [1]:
Y_pred = model.predict(X_valid).clip(0, 20)

## Отрисовка графика для наглядного сравнения

In [1]:
submission = pd.DataFrame({
    "ID": X_valid.index, 
    "item_cnt_month": Y_pred
})
x = submission['ID'].head(200)
y_pred = submission['item_cnt_month'].head(200)
y_true = Y_valid.head(200)
fig = plt.figure(figsize=(25, 5))
ax = fig.add_subplot(111)
ax.plot(x, y_pred, label = 'Предсказанные значения', color = 'red')
ax.plot(x, y_true, label = 'Истинные', color = 'green')

ax.legend()

plt.show()

## Подсчитаем ошибку на валидационной выборке, сравнивая с истинными значениями

Метрика которая использовалась для расчета ошибки : Root Mean Squared Error (**RMSE**)

 $$RMSE = \sqrt{\frac{1}{n}  \sum{(y - y_{pred})^2}}$$

Лучшая модель - модель с наименьшей RMSE на тестовой выборке

In [1]:
print(f'MSE на валиционной выборке: {round(mse(Y_valid, Y_pred), 5)}\nRMSE на валиционной выборке: {round(math.sqrt(mse(Y_valid, Y_pred)),5)}')

Ошибка довольно низкая, что не может не радовать

## Выведем график оценки важности признаков

In [1]:
def plot_feature_importance(importance,names,model_type):

  feature_importance = np.array(importance)
  feature_names = np.array(names)

  data={'feature_names':feature_names,'feature_importance':feature_importance}
  fi_df = pd.DataFrame(data)

  fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

  plt.figure(figsize=(20,12))
  sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
  plt.title('Оценка важности признаков ' + model_type)
  plt.xlabel('Важность признака')
  plt.ylabel('Признак')

In [1]:
plot_feature_importance(model.get_feature_importance(),X_train.columns,'CATBOOST')

Самое сильное влияение у цены товара

Признаки показывающие сдвиг оказывают одно из сильнейших влияений на модель

# Предскажем объем продаж за Ноябрь 2015

In [1]:
Y_test = model.predict(X_test).clip(0, 20)

Сохраним в файл

In [1]:
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('submission.csv', index=False)

In [1]:
submission